
# Workflow Practice

In this notebook, you’ll practice connecting to a SQLite database, creating tables from CSV files using Pandas, and writing SQL queries to explore the data.

The dataset comes from the [Bike Store Sample Database](https://www.kaggle.com/datasets/dillonmyrick/bike-store-sample-database) by Dillon Myrick. It models a fictional bike retailer with multiple stores, products, customers, and staff. Each table connects to others using foreign keys such as `customer_id`, `store_id`, and `product_id`.

You’ll:
- Connect to a local SQLite database
- Create tables using `pandas.to_sql()`
- Write and test SQL queries using `pd.read_sql()`

All of your work will take place directly in this notebook. Each question prompt is written below as a Markdown cell, followed by an empty code cell for you to write your query.



## Step 1: Connect to the Database

Run the following cell to connect to (or create) a SQLite database called `bike_store.db`.  
If the file doesn’t exist yet, SQLite will automatically create it.


In [1]:
import sqlite3
import pandas as pd

In [2]:
connection = sqlite3.connect("bike_store.db")
connection


## Step 2: Create Tables from CSV Files

The `data/` folder contains one CSV file per table.  
Use `pandas.read_csv()` and `DataFrame.to_sql()` to load each file into your database.

You only need to do this once.  
After that, you’ll be able to run queries against your newly created tables.


In [3]:
# Example for one file
customers = pd.read_csv("data/customers.csv")
customers.to_sql("customers", connection, if_exists="replace", index=False)

1445

In [4]:
# Repeat for all other files in the data folder, or use a loop.
import pandas as pd
import sqlite3
from pathlib import Path

connection = sqlite3.connect("bike_store.db")
data_folder = Path("data")

for file in data_folder.glob("*.csv"):
    table_name = file.stem  # name without .csv
    df = pd.read_csv(file)
    df.to_sql(table_name, connection, if_exists="replace", index=False)
    print(f"Loaded {file.name} into table '{table_name}'")


Loaded brands.csv into table 'brands'
Loaded categories.csv into table 'categories'
Loaded customers.csv into table 'customers'
Loaded orders.csv into table 'orders'
Loaded order_items.csv into table 'order_items'
Loaded products.csv into table 'products'
Loaded staffs.csv into table 'staffs'
Loaded stocks.csv into table 'stocks'
Loaded stores.csv into table 'stores'


### Verify Your Tables

Run a query to make sure your tables were created successfully.

In [5]:

pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", connection)


name
0       brands
1   categories
2    customers
3       orders
4  order_items
5     products
6       staffs
7       stocks
8       stores

## Step 3: Test a Simple Query

Before starting the exercises, confirm your connection and tables are working by previewing the first few rows of the `customers` table.

In [6]:

pd.read_sql("SELECT * FROM customers LIMIT 5;", connection)


customer_id  first_name last_name           phone                    email  \
0            1       Debra     Burks            None    debra.burks@yahoo.com   
1            2       Kasha      Todd            None     kasha.todd@yahoo.com   
2            3      Tameka    Fisher            None    tameka.fisher@aol.com   
3            4       Daryl    Spence            None     daryl.spence@aol.com   
4            5  Charolette      Rice  (916) 381-6003  charolette.rice@msn.com   

                  street           city state  zip_code  
0      9273 Thorne Ave.    Orchard Park    NY     14127  
1       910 Vine Street        Campbell    CA     95008  
2  769C Honey Creek St.   Redondo Beach    CA     90278  
3        988 Pearl Lane       Uniondale    NY     11553  
4         107 River Dr.      Sacramento    CA     95820

### Q1. List all customers and their cities.

Return the first name, last name, and city of each customer. Sort alphabetically by last name and then by first name.

In [7]:
# Your query here
pd.read_sql("SELECT first_name, last_name, city FROM customers ORDER BY last_name, first_name;", connection)


first_name  last_name                 city
0          Ester    Acevedo          San Lorenzo
1         Jamika    Acevedo           Ozone Park
2          Penny    Acevedo         Ballston Spa
3       Bettyann     Acosta            Lancaster
4          Shery     Acosta     Saratoga Springs
...          ...        ...                  ...
1440        Edda      Young      North Tonawanda
1441      Jasmin      Young              Helotes
1442  Alexandria     Zamora          Schenectady
1443       Jayme     Zamora  Springfield Gardens
1444       Ollie  Zimmerman              Anaheim

[1445 rows x 3 columns]

### Q2. Show all products and their prices.

Display each product name along with its list price. Sort by price in descending order.

In [8]:
# Your query here
pd.read_sql("SELECT product_name, list_price FROM products ORDER BY list_price DESC;", connection)

product_name  list_price
0             Trek Domane SLR 9 Disc - 2018    11999.99
1             Trek Domane SLR 8 Disc - 2018     7499.99
2          Trek Silque SLR 8 Women's - 2017     6499.99
3            Trek Domane SL Frameset - 2018     6499.99
4    Trek Domane SL Frameset Women's - 2018     6499.99
..                                      ...         ...
316                    Trek Kickster - 2018      159.99
317         Trek Boy's Kickster - 2015/2017      149.99
318             Trek Girl's Kickster - 2017      149.99
319          Sun Bicycles Lil Kitt'n - 2017      109.99
320  Strider Classic 12 Balance Bike - 2018       89.99

[321 rows x 2 columns]

### Q3. Find all customers from California.

Return first name, last name, city, and state for all customers whose state is 'CA'. Sort alphabetically by last name.

In [9]:
# Your query here
pd.read_sql("SELECT first_name, last_name, city, state FROM customers WHERE state = 'CA' ORDER BY last_name, first_name;", connection)

first_name  last_name                    city state
0        Ester    Acevedo             San Lorenzo    CA
1       Jamaal     Albert                Torrance    CA
2        Sindy   Anderson                  Pomona    CA
3        Twana     Arnold                 Anaheim    CA
4       Selene     Austin                  Duarte    CA
..         ...        ...                     ...   ...
279     Darren       Witt               Coachella    CA
280       Lucy      Woods  Palos Verdes Peninsula    CA
281       Joel       Wynn               San Diego    CA
282      Yvone      Yates               San Pablo    CA
283      Ollie  Zimmerman                 Anaheim    CA

[284 rows x 4 columns]

### Q4. Count how many products are in each category.

Return the category name and the number of products in that category. Sort from the highest count to the lowest.

In [10]:
# Your query here
pd.read_sql("SELECT c.category_name, COUNT(p.product_id) AS product_count FROM categories AS c JOIN products AS p ON c.category_id = p.category_id GROUP BY c.category_name ORDER BY product_count DESC;", connection)

category_name  product_count
0    Cruisers Bicycles             78
1           Road Bikes             60
2       Mountain Bikes             60
3    Children Bicycles             59
4     Comfort Bicycles             30
5       Electric Bikes             24
6  Cyclocross Bicycles             10

### Q5. Find all orders placed in 2018.

List the order ID, order date, and customer ID for orders made during the year 2018. Sort by order date.

In [11]:
# Your query here
pd.read_sql("SELECT order_id, order_date, customer_id FROM orders WHERE strftime('%Y', order_date) = '2018' ORDER BY order_date;", connection)

order_id  order_date  customer_id
0        1324  2018-01-01          862
1        1325  2018-01-01           68
2        1326  2018-01-01          567
3        1327  2018-01-02         1026
4        1328  2018-01-02         1083
..        ...         ...          ...
287      1611  2018-09-06            6
288      1612  2018-10-21            3
289      1613  2018-11-18            1
290      1614  2018-11-28          135
291      1615  2018-12-28          136

[292 rows x 3 columns]

### Q6. Show each order with its total number of items.

Join the `orders` and `order_items` tables. Group by order ID and return the number of items per order.

In [12]:
# Your query here
pd.read_sql("""
SELECT o.order_id,
       COALESCE(SUM(oi.quantity), 0) AS total_items
FROM orders AS o
LEFT JOIN order_items AS oi
  ON oi.order_id = o.order_id
GROUP BY o.order_id
ORDER BY o.order_id;
""", connection)

order_id  total_items
0            1            8
1            2            3
2            3            2
3            4            2
4            5            4
...        ...          ...
1610      1611            4
1611      1612            8
1612      1613            3
1613      1614            5
1614      1615            4

[1615 rows x 2 columns]

### Q7. List total revenue per store.

Revenue = quantity * list_price * (1 - discount). Join `orders`, `order_items`, and `stores`, group by store name, and return total revenue.

In [13]:
# Your query here
pd.read_sql("""
SELECT s.store_name,
       ROUND(SUM(oi.quantity * oi.list_price * (1 - oi.discount)), 2) AS total_revenue
FROM stores s
JOIN orders o      ON o.store_id = s.store_id
JOIN order_items oi ON oi.order_id = o.order_id
GROUP BY s.store_id, s.store_name
ORDER BY total_revenue DESC, s.store_name;
""", connection)

store_name  total_revenue
0     Baldwin Bikes     5215751.28
1  Santa Cruz Bikes     1605823.04
2     Rowlett Bikes      867542.24

### Q8. Find the top 5 customers who spent the most overall.

Join `customers`, `orders`, and `order_items`. Sum the total spending per customer and return the top five spenders.

In [14]:
# Your query here
pd.read_sql("""
SELECT c.first_name,
       c.last_name,
       ROUND(SUM(oi.quantity * oi.list_price * (1 - oi.discount)), 2) AS total_spend
FROM customers c
JOIN orders o       ON o.customer_id = c.customer_id
JOIN order_items oi ON oi.order_id   = o.order_id
GROUP BY c.customer_id, c.first_name, c.last_name
ORDER BY total_spend DESC, c.last_name, c.first_name
LIMIT 5;
""", connection)

first_name last_name  total_spend
0     Sharyn   Hopkins     34807.94
1    Pamelia    Newman     33634.26
2       Abby    Gamble     32803.01
3    Lyndsey      Bean     32675.07
4     Emmitt   Sanchez     31925.89

### Q9. Show the best-selling product in each category.

Join `products`, `order_items`, and `categories`. For each category, identify the product with the highest total quantity sold.

In [15]:
# Your query here
pd.read_sql("""
WITH sales AS (
  SELECT c.category_id,
         c.category_name,
         p.product_id,
         p.product_name,
         SUM(oi.quantity) AS qty_sold
  FROM categories c
  JOIN products p     ON p.category_id = c.category_id
  JOIN order_items oi ON oi.product_id = p.product_id
  GROUP BY c.category_id, c.category_name, p.product_id, p.product_name
)
SELECT category_name, product_name, qty_sold
FROM (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY category_id ORDER BY qty_sold DESC, product_name) AS rn
  FROM sales
) ranked
WHERE rn = 1
ORDER BY category_name;
""", connection)

category_name                                   product_name  \
0    Children Bicycles  Electra Girl's Hawaii 1 (20-inch) - 2015/2016   
1     Comfort Bicycles         Electra Townie Original 7D - 2015/2016   
2    Cruisers Bicycles             Electra Cruiser 1 (24-Inch) - 2016   
3  Cyclocross Bicycles                    Surly Straggler 650b - 2016   
4       Electric Bikes                           Trek Conduit+ - 2016   
5       Mountain Bikes          Surly Ice Cream Truck Frameset - 2016   
6           Road Bikes                  Trek Domane SLR 6 Disc - 2017   

   qty_sold  
0       154  
1       148  
2       157  
3       151  
4       145  
5       167  
6        43

### Q10. Identify the employees (staff) who processed the most orders.

Join `staffs` and `orders`. Count the number of orders handled by each staff member and return the results sorted by highest total.

In [16]:
# Your query here
pd.read_sql("""
SELECT s.first_name,
       s.last_name,
       COUNT(o.order_id) AS orders_processed
FROM staffs s
LEFT JOIN orders o
  ON o.staff_id = s.staff_id
GROUP BY s.staff_id, s.first_name, s.last_name
ORDER BY orders_processed DESC;
""", connection)

first_name last_name  orders_processed
0   Marcelene     Boyer               553
1      Venita    Daniel               540
2       Genna   Serrano               184
3      Mireya  Copeland               164
4        Kali    Vargas                88
5       Layla   Terrell                86
6     Fabiola   Jackson                 0
7      Virgie   Wiggins                 0
8    Jannette     David                 0
9  Bernardine   Houston                 0